In [1]:
import tensorflow as tf
from tqdm import tqdm
import keras
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten, Softmax, Input, Reshape,Lambda,\
    Conv2D, MaxPool2D, Dropout,Concatenate,Embedding,BatchNormalization
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import re
import numpy as np
import os
import time
import json
import inception_v3
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
fp2="data/senticap_dataset.json"

with open(fp2,'r') as f2:
    senticap_data= json.load(f2)



In [3]:

        
senticap_list=[]
senticap_list_raw=[]
l=0
for img_obj in senticap_data['images']:
    # if (l>20):
    #     break
    # l+=1
    img_name= img_obj['filename']
    for sent in img_obj['sentences']:
        sentiment= sent['sentiment']
        raw= sent['raw'].lower()
        w0= raw.rstrip()
        w0 = [x.replace(".", "") for x in w0]
        w0= [x.replace(",","") for x in w0]
        w0= [x.replace("'","") for x in w0]
        w0= [x.replace("?","") for x in w0]
        w0="".join(w0)
        senticap_list.append((img_name,sentiment,w0))
        senticap_list_raw.append((img_name,w0))
        
        

In [61]:
"""write to cap file for nlp treebank"""
filename='new_cap.txt'
with open(filename,'w') as f4:
    for item in senticap_list_raw:
        w0= item[1]
        w0= w0.lower()

        f4.write(w0+" .\n")
        

In [4]:
"""read from output of nlp treebank"""

fname="new_out.txt"
i=0
l1=10000
senti_score=[]
senti_avg=[]
error=0
senti_var=[]
train_caption=[]
senti_cap_dict={}
with open(fname) as f1:
    while(i<l1):
        w1= f1.readline().strip()
        if not w1:
            break
        w2= f1.readline().strip()
        w1 = [x.replace(".", "") for x in w1]
        w1= [x.replace(",","") for x in w1]
        w1= [x.replace("'","") for x in w1]
        w1= [x.replace("?","") for x in w1]
        w1= "".join(x for x in w1)
        w1= w1.rstrip()
        w1= "".join(x for x in w1)

        if w2.islower():
            w1=w2
            w2=f1.readline().strip()
        a= 0
        if w2=="Negative":
            a= -1
        elif w2=="Positive":
            a= 1
        elif w2=="Very negative":
            a= -1
        elif w2=="Very positive":
            a= 1
        elif w2=="Neutral":
            a= 0
        else:
            a=0
            error+=1
            print(w2)
        senti_score.append(a)
        train_caption.append(w1)
        senti_cap_dict[w1]=a
        i+=1
# print(senti_score[0:500] )
print(len(train_caption),"train-cap_length")
print(len(senti_score),"train-senti_score")

" .
8861 train-cap_length
8861 train-senti_score


In [5]:
PATH_DIR= '/home/yiyang/project/caption/caption'
PATH= PATH_DIR+'/data/val2014/'

In [6]:
image_vec=[]
sentiment_vec=[]
caption_vec=[]

for item in senticap_list:
    img= item[0]
    if item[1]==1:
        senti=1
    elif item[1]==0:
        senti=-1
    else:
        senti=0
        
    caption= item[2]
    # print(caption+"#")
    if caption in senti_cap_dict.keys():
        new_senti= senti_cap_dict[caption]
    else:
        new_senti= 0
    img= PATH + img 
    caption= '<start> ' + caption + ' <end>'
    image_vec.append(img)
    sentiment_vec.append(new_senti)
    caption_vec.append(caption)
    

In [7]:
num_examples = 8861
start=0

image_name_train= image_vec[:num_examples]
senti_train= sentiment_vec[:num_examples]
caption_train= caption_vec[:num_examples]
print(caption_train[8000])

<start> a plane sitting on the runway in a clear field during a beautiful sunset <end>


In [8]:
print(image_name_train[0])
# img=load_image(image_name_train[0])
img=cv2.imread(image_name_train[0]) 
print(caption_train[0])
# img=cv2.imshow('image',img)
cv2.imshow('ImageWindow', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


/home/yiyang/project/caption/caption/data/val2014/COCO_val2014_000000389081.jpg
<start> a plate of delicious food including french fries <end>


In [9]:
def load_image(image_path):
    img= cv2.imread(image_path)
    img= cv2.resize(img,(299,299))
    return img


In [10]:
units= 512
embedding_size=512
BATCH_SIZE=16
vocab_size=4500
max_length=20

In [11]:
top_k = vocab_size-1
tokenizer = keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(caption_train)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


train_seqs = tokenizer.texts_to_sequences(caption_train)
cap_vector_train = keras.preprocessing.sequence.pad_sequences(train_seqs,dtype='int32', padding='post',truncating='post',maxlen=max_length,value=0)

print(train_seqs[0])
print(cap_vector_train[0])



[3, 2, 117, 6, 201, 32, 764, 835, 703, 4]
[  3   2 117   6 201  32 764 835 703   4   0   0   0   0   0   0   0   0
   0   0]


In [12]:

model=inception_v3.InceptionV3(include_top=True,weights=None)
weights_path = 'data/image_net.h5'
# image_model = tf.keras.applications.InceptionV3(include_top=False,
#                                                 weights='imagenet')
model.load_weights(weights_path)
print(model.output_shape)
for layer in model.layers[:312]:
    layer.trainable = False
    

W0928 18:29:50.007477 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



W0928 18:29:50.023572 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



W0928 18:29:50.026895 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W0928 18:29:50.042816 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



W0928 18:29:50.043344 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



W0928 18:29:51.519211 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



W0928 18:29:51.636298 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



W0928 18:29:51.990508 139620787287872 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



(None, 1000)


In [13]:

new_input = model.layers[0].input
hidden_layer = model.get_layer('avg_pool').output

image_model = keras.Model(new_input, hidden_layer)
print(image_model.output_shape)

(None, 2048)


In [16]:
def model_train(vocab_size, max_len, units):
    image_input=Input(shape=(299,299,3),dtype=tf.float32)
    seq_input= Input(shape=(max_len,),dtype=tf.int32)
    senti_input= Input(shape=(1,),dtype=tf.int32)
    
    image_feature=image_model(image_input)
    image_feature=Dropout(rate=0.5)(image_feature)
    
    embedded_image= Dense(embedding_size,name="img_embedding",use_bias=False)(image_feature)
    # print(embedded_image.shape,"embedded_img")
    
    embedded_image= BatchNormalization()(embedded_image)
    
    expand_dim= Lambda (lambda w: tf.expand_dims(w, 1))
    embedded_image=expand_dim(embedded_image)
    

    embedded_seq= Embedding(input_dim=vocab_size,output_dim=units,mask_zero=True,name="seq_input_embedding")(seq_input)
    # print(embedded_seq.shape,"seq_shape")
    embedded_seq=Dropout(rate=0.5)(embedded_seq)
    
    output_embedding=Dense(vocab_size,activation='softmax')
    softmax_layer=Lambda (lambda w: tf.argmax(w,1))
    
    LSTM_layer= keras.layers.LSTM(units, return_sequences=True, 
                             return_state=True,
                             recurrent_initializer='glorot_uniform',
                             dropout=0.5,name="lstm_layer")


    output,state_c, state_h = LSTM_layer(embedded_image)
    
    lstm_output , output, state_h= LSTM_layer(embedded_seq,initial_state=[state_c,state_h])
    # print(lstm_output.shape,"lstm_output")
    logits= output_embedding(lstm_output)
    # print(logits.shape,"shape_logit")
    remove_last= Lambda (lambda w: tf.slice(w,[0,0,0],[-1,max_length-1,-1]))
    logits_2= remove_last(logits)
    # print(logits_2.shape,"logit_shape")
    # logits_3= Reshape(target_shape=(512,))(logits_2)
    # print(logits_3.shape,"stack lstm")
    train_model= keras.Model([image_input,seq_input,senti_input],logits_2)
    train_model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam')

    # print(train_model.output_shape)
    
    return train_model



In [17]:
model_train_1= model_train(vocab_size=vocab_size,max_len=max_length,units=units)

In [18]:
def model_inference(vocab_size, max_len, units):

    image_input=Input(shape=(299,299,3),dtype=tf.float32)
    seq_input= Input(shape=(1,),dtype=tf.int32)
    senti_input= Input(shape=(1,),dtype=tf.int32)
    
    output_embedding= Dense(vocab_size,activation='softmax')
    
    image_feature=image_model(image_input) 
    embedded_image= Dense(embedding_size,use_bias=False,name="img_embedding")(image_feature)
    embedded_image= BatchNormalization()(embedded_image)
    expand_dim= Lambda (lambda w: tf.expand_dims(w, 1))
    embedded_image=expand_dim(embedded_image)
    
    # print(embedded_image.shape,"embedded_img")
    
    embedded_seq= Embedding(input_dim=vocab_size,output_dim=units,mask_zero=True,name="seq_input_embedding")(seq_input)
    # print(embedded_seq.shape,"seq_shape")
    
    
    argmax_layer= Lambda (lambda w: tf.argmax(w,-1))
    # print(embedded_image.shape,"image_shape")
    
    
    LSTM_unit= keras.layers.LSTM(units, return_sequences=False, 
                                 return_state=True,
                                 recurrent_initializer='glorot_uniform',
                                 dropout=0.5,name="lstm_layer")
    
    
    output,state_c, state_h = LSTM_unit(embedded_image)
    # print(output.shape,"output")
    # print(state_c.shape,"state_c")
    # print(state_h.shape,"state_h")
    
    # squeeze= Lambda (lambda w: tf.squeeze(w, 1))
    
    outputs=[]
    for j in range(max_len):
        output, state_c, state_h= LSTM_unit(embedded_seq,initial_state=[output,state_h])
        
        expand_output= expand_dim(output)
        # print(output.shape,"lstm_out")
        # print(expand_output.shape,"expand")
        softmax_output= output_embedding(expand_output)
        # print(softmax_output.shape,"softmax")
        next_input= argmax_layer(softmax_output)
        outputs.append(next_input)
        # print(next_input.shape,"next input shape")
        # next_input= expand_dim(next_input)
        
        embedded_seq= Embedding(input_dim=vocab_size,output_dim=units)(next_input)
        
        # print(next_input.shape,"next input shape")
        
        
        # print(embedded_seq.shape,"loop seq shape")
        # state_c= output
    
    # print(outputs[0].shape,"output shape")
    
    inference_model= keras.Model([image_input,seq_input,senti_input],outputs)
    
    
    # print(inference_model.output_shape)
    return inference_model

In [19]:
model_inf_1= model_inference(vocab_size,max_length,units)


In [20]:
input_imgs=[]
start=0
for j in tqdm(range(8000)):
    img= load_image( image_name_train[start+j])
    input_imgs.append(img)
input_imgs= np.array(input_imgs)
input_sentis= np.array(senti_train[:8000])
input_sequences= np.array(cap_vector_train[:8000])

print(input_sequences.shape,"input_seq")
target= input_sequences[:,1:]

target= np.expand_dims(target,axis=-1)
print(target.shape,"target")


  0%|          | 0/8000 [00:00<?, ?it/s]

  0%|          | 24/8000 [00:00<00:33, 238.99it/s]

  1%|          | 50/8000 [00:00<00:32, 243.94it/s]

  1%|          | 69/8000 [00:00<00:35, 224.78it/s]

  1%|          | 87/8000 [00:00<00:37, 208.57it/s]

...

 99%|█████████▉| 7945/8000 [00:32<00:00, 233.31it/s]

100%|█████████▉| 7970/8000 [00:32<00:00, 238.03it/s]

100%|█████████▉| 7994/8000 [00:33<00:00, 234.86it/s]

100%|██████████| 8000/8000 [00:33<00:00, 241.91it/s]

(8000, 20) input_seq
(8000, 19, 1) target


In [21]:
model_train_1.fit(x=[input_imgs,input_sequences,input_sentis],y=target, batch_size=BATCH_SIZE,verbose=1,epochs=10)

model_train_1.save_weights("models/model5.h5")


Epoch 1/10


  16/8000 [..............................] - ETA: 1:10:23 - loss: 8.4127

  32/8000 [..............................] - ETA: 35:24 - loss: 8.4061  

  48/8000 [..............................] - ETA: 23:44 - loss: 8.4012

  64/8000 [..............................] - ETA: 17:54 - loss: 8.3942

...

7936/8000 [============================>.] - ETA: 0s - loss: 1.4897

7952/8000 [============================>.] - ETA: 0s - loss: 1.4894

7968/8000 [============================>.] - ETA: 0s - loss: 1.4894

7984/8000 [============================>.] - ETA: 0s - loss: 1.4888

8000/8000 [==============================] - 33s 4ms/step - loss: 1.4883


In [30]:
model_train_1.load_weights("models/model5.h5")
model_train_1.fit(x=[input_imgs,input_sequences,input_sentis],y=target, batch_size=BATCH_SIZE,verbose=2,epochs=60)

model_train_1.save_weights("models/model5.h5")


Epoch 1/60


 - 32s - loss: 0.8817


Epoch 2/60


 - 32s - loss: 0.8455


Epoch 3/60


...

 - 32s - loss: 0.4108


Epoch 59/60


 - 32s - loss: 0.4082


Epoch 60/60


 - 32s - loss: 0.4069


In [31]:
test_imgs=[]
start=0
for j in tqdm(range(8000,8200)):
    img= load_image( image_name_train[start+j])
    test_imgs.append(img)
test_imgs= np.array(test_imgs)
test_sentis= np.array(senti_train[8000:8200])
test_sequences= np.array(cap_vector_train[8000:8200])
test_seq= np.ones(shape=(200,1))
test_seq= 3* test_seq
test_out=[]


  0%|          | 0/200 [00:00<?, ?it/s]

 12%|█▏        | 24/200 [00:00<00:00, 230.78it/s]

 24%|██▍       | 49/200 [00:00<00:00, 233.82it/s]

 36%|███▋      | 73/200 [00:00<00:00, 233.28it/s]

 47%|████▋     | 94/200 [00:00<00:00, 224.64it/s]

 58%|█████▊    | 116/200 [00:00<00:00, 221.45it/s]

 71%|███████   | 142/200 [00:00<00:00, 230.37it/s]

 82%|████████▎ | 165/200 [00:00<00:00, 228.83it/s]

 95%|█████████▌| 190/200 [00:00<00:00, 233.15it/s]

100%|██████████| 200/200 [00:00<00:00, 233.67it/s]

In [32]:
model_inf_1.load_weights("models/model5.h5",by_name=True,skip_mismatch=True)
preds= model_inf_1.predict(x=[test_imgs[:200],test_seq[:200],test_sentis[:200]],batch_size=1,verbose=1)


  1/200 [..............................] - ETA: 6s

  3/200 [..............................] - ETA: 6s

  5/200 [..............................] - ETA: 6s

  7/200 [>.............................] - ETA: 6s

  9/200 [>.............................] - ETA: 6s

...

193/200 [===========================>..] - ETA: 0s

195/200 [============================>.] - ETA: 0s

197/200 [============================>.] - ETA: 0s

199/200 [============================>.] - ETA: 0s

200/200 [==============================] - 6s 30ms/step


In [ ]:
print(preds[0].shape)
preds1= np.array(preds)
print(preds1.shape)
preds2= np.squeeze(preds1,axis=2)
preds_t= np.transpose(preds2)
print(preds_t.shape)
# print(test_seq[:10])
# print(preds_t)
len1=200

sent=[]
for i in range(len1):
    words= [tokenizer.index_word[x] for x in preds_t[i]]
    sent.append(words)
print(sent[:20])


(200, 1)
(20, 200, 1)
(200, 20)
[['bump', 'contestant', 'retriever', 'uploaded', 'holding', 'butterfly', 'marks', 'disabled', 'dig', 'dig', 'dig', 'dig', 'dig', 'classic', 'classic', 'classic', 'classic', 'classic', 'classic', 'classic'], ['bump', 'contestant', 'retriever', 'uploaded', 'holding', 'butterfly', 'marks', 'disabled', 'dig', 'dig', 'dig', 'dig', 'dig', 'classic', 'classic', 'classic', 'classic', 'classic', 'classic', 'classic'], ['bump', 'mopeds', 'terminal', 'holding', 'holding', 'marks', 'puppy', 'logs', 'logs', 'dig', 'logs', 'logs', 'logs', 'loses', 'cross', 'cross', 'cross', 'logs', 'logs', 'logs'], ['bump', 'mopeds', 'terminal', 'holding', 'holding', 'marks', 'puppy', 'logs', 'logs', 'dig', 'logs', 'logs', 'logs', 'loses', 'cross', 'cross', 'cross', 'logs', 'logs', 'logs'], ['bump', 'mopeds', 'terminal', 'holding', 'holding', 'marks', 'puppy', 'logs', 'logs', 'dig', 'logs', 'logs', 'logs', 'loses', 'cross', 'cross', 'cross', 'logs', 'logs', 'logs'], ['bump', 'contesta